In [2]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_excel('../data/19-23 Cases by day.xlsx')
df = df.rename(columns={'Date': '发病日期', 'Case': '病例数'})
df

,发病日期,病例数
0,2019-01-01,0
1,2019-01-02,0
2,2019-01-03,0
3,2019-01-04,1
4,2019-01-05,0
...,...,...
1821,2023-12-27,0
1822,2023-12-28,0
1823,2023-12-29,0
1824,2023-12-30,0


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df['发病日期'] = pd.to_datetime(df['发病日期'])
df['year'] = df['发病日期'].dt.year
df['day_of_year'] = df['发病日期'].dt.dayofyear

df['adjusted_angle'] = df.apply(lambda x: 360 * (x['day_of_year'] - 1) / (366 if x['发病日期'].is_leap_year else 365), axis=1)

results = []
for year in df['year'].unique():
    yearly_data = df[df['year'] == year]
    radians = np.deg2rad(yearly_data['adjusted_angle'])
    weights = yearly_data['病例数']
    
    if weights.isnull().any() or radians.isnull().any():
        continue  

    sum_sin = np.sum(weights * np.sin(radians))
    sum_cos = np.sum(weights * np.cos(radians))
    mean_angle = np.arctan2(sum_sin, sum_cos)

    mean_vector_length = np.sqrt(sum_sin**2 + sum_cos**2) / np.sum(weights)
    weighted_circular_std = np.sqrt(-2 * np.log(mean_vector_length))

    R = np.sum(weights * np.exp(1j * radians))
    rayleigh_z = (np.abs(R) ** 2) / np.sum(weights)
    rayleigh_p = np.exp(-rayleigh_z * np.sum(weights))

    mean_angle_deg = np.rad2deg(mean_angle) % 360
    std_dev_deg = np.rad2deg(weighted_circular_std)

    peak_day_angle = (mean_angle_deg / 360) * (366 if pd.Timestamp(year=year, month=1, day=1).is_leap_year else 365)
    start_peak_angle = ((mean_angle_deg - std_dev_deg) % 360) / 360
    end_peak_angle = ((mean_angle_deg + std_dev_deg) % 360) / 360


    try:
        peak_day_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(peak_day_angle))
        start_peak_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(start_peak_angle * (366 if pd.Timestamp(year=year, month=1, day=1).is_leap_year else 365)))
        end_peak_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(end_peak_angle * (366 if pd.Timestamp(year=year, month=1, day=1).is_leap_year else 365)))
    except ValueError:
        continue 

    results.append({
        'Year': year,
        'Mean Angle (degrees)': mean_angle_deg,
        'Std Dev (degrees)': std_dev_deg,
        'Rayleigh Z': rayleigh_z,
        'Rayleigh P-value': rayleigh_p,
        'Peak Day': peak_day_date,
        'Start of Peak Period': start_peak_date,
        'End of Peak Period': end_peak_date
    })

results_df = pd.DataFrame(results)
results_df


,Year,Mean Angle (degrees),Std Dev (degrees),Rayleigh Z,Rayleigh P-value,Peak Day,Start of Peak Period,End of Peak Period
0,2019,180.200999,68.756807,104.950770,0.0,2019-07-02,2019-04-23,2019-09-10
1,2020,167.800239,56.999544,238.255422,0.0,2020-06-19,2020-04-22,2020-08-16
2,2021,176.808816,66.585214,140.690205,0.0,2021-06-29,2021-04-22,2021-09-04
3,2022,162.621546,57.572311,306.407407,0.0,2022-06-14,2022-04-17,2022-08-12
4,2023,174.420447,64.486921,398.662064,0.0,2023-06-26,2023-04-22,2023-08-31


In [ ]:


df['发病日期'] = pd.to_datetime(df['发病日期'])
df['year'] = df['发病日期'].dt.year
df['month'] = df['发病日期'].dt.month
df['day_of_year'] = df['发病日期'].dt.dayofyear


results = []
for year in df['year'].unique():
    yearly_data = df[df['year'] == year]
    monthly_cases = yearly_data.groupby('month')['病例数'].sum()
    r = monthly_cases / monthly_cases.sum()  

    R_x = (r.get(2, 0) + r.get(6, 0) - r.get(8, 0) - r.get(12, 0)) / 2 \
          + (np.sqrt(3) / 2) * (r.get(3, 0) + r.get(5, 0) - r.get(9, 0) - r.get(11, 0)) \
          + (r.get(4, 0) - r.get(10, 0))

    R_y = (r.get(3, 0) - r.get(5, 0) - r.get(9, 0) + r.get(11, 0)) / 2 \
          + (np.sqrt(3) / 2) * (r.get(2, 0) - r.get(6, 0) - r.get(8, 0) + r.get(12, 0)) \
          + (r.get(1, 0) - r.get(7, 0))


    M = np.sqrt(R_x**2 + R_y**2)


    results.append({
        'Year': year,
        'Rx': R_x,
        'Ry': R_y,
        'Concentration (M)': M
    })


results_df = pd.DataFrame(results)
results_df


,Year,Rx,Ry,Concentration (M)
0,2019,0.103853,-0.482718,0.493764
1,2020,0.259686,-0.547402,0.605876
2,2021,0.133134,-0.483874,0.501855
3,2022,0.297777,-0.516720,0.596381
4,2023,0.160116,-0.500366,0.525360
